In [336]:
import pandas as pd
import requests
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import pylab as pl
%pylab inline

from API_Key import get_key
import argparse
import json
import pprint
import requests
import sys
import urllib
import geocoder
from pandana.loaders import osm
from shapely.geometry import Point
import mplleaflet as mpll


try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

Populating the interactive namespace from numpy and matplotlib


In [333]:
def get_APIdata(key, place = 'Ulaanbaatar', category = 'establishment', strictbound=True):
    
    """
    Inputs:
    
    key: Google API Key
    
    string: str: category of data to be queried e.g. 'hospoital', 'museum' etc.
    
    place: str: Area of Interest
    
    category: str: establishment or geocode
    
    // establishment returns only businessnes
    
    strictbound: bool: Hard restriction on radius for data search
    
    Returns:
    Json dump
    
    """
    amenities = ['hospital', 'dentist', 'doctor', 'restaurant','cafe','school','bank','pharmacy','park']
    string=None
    session_id = np.random.randint(1,99999999)
    country = geocoder.arcgis("{}".format(place))
        
    radius= 100000
    
    lat = country.geojson['features'][0]['geometry']['coordinates'][1]
    lon = country.geojson['features'][0]['geometry']['coordinates'][0]
    
    if strictbound:
        url = "https://maps.googleapis.com/maps/api/place/autocomplete/json?input={string}&\
                    types={category}&location={y},{x}&radius={rad}&key={key}&strictbounds&sessiontoken={sess}".format\
                        (string=string, category=category,y=lat, x=lon, rad=radius, key=key, sess=session_id)
    else:
        url = "https://maps.googleapis.com/maps/api/place/autocomplete/json?input={string}&\
                    types={category}&location={y},{x}&radius={rad}&key={key}&strictbounds&sessiontoken={sess}".format\
                        (string=string, category=category,y=lat, x=lon, rad=radius, key=key, sess=session_id)
    
    res_list = []
    
    for i in amenities:
        link = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={y},{x}\
        &radius={rad}&type={string}&key={key}".format(y=lat, x=lon, rad=radius, string=i, key=key)


        response = requests.request('GET', link)
        res_list.append(response.json())

    return res_list

In [300]:
reqs = get_APIdata(get_key())

In [335]:
def get_info(reqs):
    """
    Input:
    reqs: json response from API query
    
    Returns:
    Pandas Dataframe
    """
    entity, city, pl_id, lat,entity_type_1,entity_type_2, entity_type_3, lon = [],[],[],[],[],[],[],[]
    
    for j in reqs:
        for i in j['results']:
            entity.append(i['name'])
            city.append(i['vicinity'])
            lat.append(i['geometry']['location']['lat'])
            lon.append(i['geometry']['location']['lng'])
            entity_type_1.append(i['types'][0])
            entity_type_2.append(i['types'][1])
            entity_type_3.append(i['types'][2])
            pl_id.append(i['place_id'])
        
    df = pd.DataFrame()
    df['Place_ID'] = pl_id
    df['Entity_name'] = entity
    df['City'] = city
    df['Entity_Type_1'] = entity_type_1
    df['Entity_Type_2'] = entity_type_2
    df['Entity_Type_3'] = entity_type_3
    df['Lat'] = lat
    df['Lon'] = lon
    
        
    return df

In [301]:
google_data = get_info(reqs)

In [311]:
google_data.Entity_Type_1.unique()

array(['hospital', 'health', 'dentist', 'hair_care', 'doctor',
       'night_club', 'lodging', 'spa', 'restaurant', 'cafe',
       'electronics_store', 'book_store', 'university', 'school', 'gym',
       'point_of_interest', 'bank', 'department_store', 'pharmacy',
       'travel_agency', 'amusement_park', 'park', 'zoo', 'campground'],
      dtype=object)

In [303]:
google_data['geometry'] = (list(zip(google_data.Lon,google_data.Lat)))
google_data['geometry'] = google_data.geometry.apply(lambda x: Point(x))
google_data = gpd.GeoDataFrame(google_data, geometry='geometry')

In [318]:
ax = google_data[google_data['Entity_Type_1'] == 'school'].plot()
mpll.display(fig=ax.figure)

In [305]:
country = geocoder.arcgis('Ulaanbaatar')
bbox = gpd.GeoDataFrame(country.bbox).T
bbox.columns=['lat','lon']
bbox['geometry'] = (list(zip(bbox.lon,bbox.lat)))
bbox['geometry'] = bbox.geometry.apply(lambda x: Point(x))
bbox.set_geometry(bbox['geometry'])
coords = [country.bbox['southwest'][0], country.bbox['southwest'][1],
          country.bbox['northeast'][0], country.bbox['northeast'][1]]

amenities = ['hospital', 'dentist', 'doctor', 'restaurant','cafe','school','bank','pharmacy','park']

osm_tags = '"amenity"~"{}"'.format('|'.join(amenities))

pois = osm.node_query(coords[0],coords[1],coords[2],coords[3],tags=osm_tags) # required: lat_min, lng_min, lat_max, lng_max
pois = pois[pois['amenity'].isin(amenities)]

In [306]:
osm_data = pois[['amenity', 'lat', 'lon', 'name', 'name:en']]

In [307]:
osm_data['geometry'] = (list(zip(osm_data.lon,osm_data.lat)))
osm_data['geometry'] = osm_data.geometry.apply(lambda x: Point(x))
osm_data = gpd.GeoDataFrame(osm_data, geometry='geometry')

/nfshome/gb1877/.conda/envs/capVE/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/nfshome/gb1877/.conda/envs/capVE/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [319]:
osm_data.head()

,amenity,lat,lon,name,name:en,geometry
id,,,,,,
478559818,bank,47.900497,106.908680,NaN,NaN,POINT (106.9086797 47.9004974)
478559820,restaurant,47.912344,106.913782,Баянгол,NaN,POINT (106.9137823 47.9123441)
551373114,bank,47.917457,106.911547,Валют солих төв,NaN,POINT (106.9115467 47.9174565)
637637033,restaurant,48.389827,106.213896,NaN,NaN,POINT (106.2138956 48.3898268)
1013903461,cafe,47.852966,106.762464,NaN,NaN,POINT (106.7624644 47.8529664)


In [308]:
ax2 = osm_data.plot()
mpll.display(fig=ax2.figure)

In [309]:
len(google_data), len(osm_data)

(180, 1417)

In [342]:
country.geojson['features'][0]['bbox']

[106.14667000000004,
 47.151150000000044,
 107.68867000000004,
 48.693150000000045]